In [1]:
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
def pointremove(image):
    
    img=image.copy()
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(img, connectivity=4)
    sizes = stats[:, -1]
    for i in range(1, nb_components):
        if sizes[i]<100 :
            img[output==i]=0
    return img

In [2]:
def maxmeanpreprop(image):
    img1 = image.copy()
    g_img = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img = g_img.copy()
    ret1,th1 = cv2.threshold(g_img,40,255,cv2.THRESH_BINARY)
    th1=pointremove(th1)
    blur = cv2.GaussianBlur(th1,(5,5),0)
    ret2,th2 = cv2.threshold(blur,130,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    a_img=th2&img
    plt.imshow(a_img)
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(a_img, connectivity=4)
    sizes = stats[:, -1]

    max_label = 0
    max_mean = 0

    for i in range(1, nb_components):
        mean=np.mean(a_img[output==i])
        if mean > max_mean and sizes[i]>1500 :
            max_label = i
            max_mean = mean

    img3 = np.zeros(output.shape)
    img3[output == max_label] = 255
    new_image = img3[stats[max_label][1]:stats[max_label][1]+stats[max_label][3],stats[max_label][0]:stats[max_label][0]+stats[max_label][2]]
    image_for_input = cv2.resize(new_image, (100, 100), interpolation = cv2.INTER_NEAREST)
    return image_for_input